# Bipartite entanglement

One of the basic concepts in quantum information is entanglement. It is a property of quantum states that is not shared by classical states. One key problem is to determine whether a given quantum state is entangled or not. This is a NP problem in general, but there are some useful criteria to detect entanglement. In this tutorial, we will introduce some basic concepts about bipartite entanglement, which means we only care about the entanglement between two different parties.


In [ ]:
import numpy as np
import matplotlib.pyplot as plt

import numqi

## Pure state

Consider a Hilbert space $\mathcal{H} = \mathcal{H}_1 \otimes \mathcal{H}_2$, a pure state $\vert \psi \rangle \in \mathcal{H}$ is *separable* iff $\vert \psi \rangle = \vert \phi_1 \rangle \otimes \vert \phi_2 \rangle$ for some pure states $\vert \phi_i \rangle \in \mathcal{H}_i$; otherwise it is called *entangled*.

A useful tool for the characterization of bipartite entanglement is the *Schmidt decomposition*: any $\vert \psi \rangle \in \mathcal{H}$ can be written as 

$$ 
|\psi\rangle=\sum_{i=1}^r \lambda_i\left|e_i\right\rangle\left|f_i\right\rangle,
$$
where the coefficients $\lambda_i$ are positive numbers that can be ordered as $\lambda_1 \geq \lambda_2 \geq \ldots \geq \lambda_r >0$, whose squares sum up to one, $\{\vert e_i\rangle\}$ and $\{\vert f_i\rangle\}$ are orthonormal bases of the local Hilbert spaces and $r \leq \min(d_1,d_2)$ is called the *Schmidt rank* of $\vert \psi \rangle$ representing the minimum terms for decomposing a state. It is equivalent to the singular value decomposition (SVD) of the corresponding coefficient matrix. In that sense, sometimes we also use the notion of *matrix rank* parallelly.

Some famous entangled states are Bell states [wiki-link](https://en.wikipedia.org/wiki/Bell_state), GHZ states [wiki-link](https://en.wikipedia.org/wiki/Greenberger%E2%80%93Horne%E2%80%93Zeilinger_state), W state [wiki-link](https://en.wikipedia.org/wiki/W_state), etc.

$$ \mathrm{Bell}: |00\rangle \pm |11\rangle, |01\rangle \pm |10\rangle  $$

$$ \mathrm{GHZ}: |000\rangle + |111\rangle $$

$$ \mathrm{W}: |001\rangle + |010\rangle + |100\rangle $$

(not normalized for conciseness). Below we can calculate the Schmidt rank (number of nonzero singular values) of these states (for three partite state, we simply divide into the first party and the others).

In [ ]:
state00 = np.array([1,0,0,0], dtype=np.float64)
state_bell = numqi.state.Bell()
state_ghz = numqi.state.GHZ()
state_w = numqi.state.W(3)

for state in [state00, state_bell, state_ghz, state_w]:
    singular_value = np.linalg.svd(state.reshape(2,-1), compute_uv=False)
    tmp0 = 'entangled' if singular_value[1] > 1e-7 else 'separable'
    print(f'[{tmp0}] singular value: {singular_value}')

Equivalently, one can check the purity of the reduced density matrix (RDM), if the purity of the RDM is less than one, then the state is entangled. The purity of a density matrix $\rho$ is defined as [wiki-link](https://en.wikipedia.org/wiki/Purity_(quantum_mechanics))

$$ \gamma=\mathrm{Tr}[\rho^2] $$

Let's verify this criterion for the above states numerically.

In [ ]:
for state in [state00, state_bell, state_ghz, state_w]:
    tmp0 = state.reshape(2, -1)
    rdm = tmp0 @ tmp0.T.conj()
    purity = numqi.utils.get_purity(rdm)
    tmp0 = 'entangled' if (1-purity>1e-7) else 'separable'
    print(f'[{tmp0}] purity: {purity}')


## Quantum subspace

Given a subspace $\mathcal{S}$ spanned by a set of bipartite states $\{ \vert \psi_1\rangle, \vert \psi_2\rangle,\dots,\vert \psi_m\rangle\}$, we define the *minimal rank* of  $\mathcal{S}$ as the following:
$$
\operatorname{r}(\mathcal{S})=\min_{\vert\psi\rangle \in \mathcal{S}}\operatorname{SR}(\vert\psi\rangle),
$$
where $\operatorname{SR}$ represents the schmidt rank. $\vert \psi \rangle \in \mathcal{S}$ means it can be derived by a combination of the states within $\mathcal{S}$, i.e.,
$$
\vert \psi \rangle=\sum_{i=1}^{m}c_i\vert \psi_i\rangle,
$$
where $c_i$'s are some complex coefficients constrained to the normalization condition. If the minimal rank of a given subspace is larger than one, we say that the subspace is entangled.

Since a bipartite state is equivalent to a matrix, the separability problem in a bipartite quantum subspace is strongly related to the problem discussed in [doc-link](../../foundation/matrix_space/space_schmidt_rank.ipynb). Interested readers can check it for more details and examples.


## Mixed state

Let $\mathcal{D}_d$ be the collection of all density matrices of a $d$-dimensional quantum system.

$$ \mathcal{D}_{d}=\left\{ \rho\in\mathbb{C}^{d\times d}:\rho\succeq0,\mathrm{Tr}[\rho]=1\right\} $$

For a bipartite quantum system $AB$, all separable density matrix (SEP) is the convex hull (conv) of the product density matrix:

$$ \mathrm{SEP}=\mathrm{conv}\left(\mathcal{D}_{A}\otimes\mathcal{D}_{B}\right) \subseteq \mathcal{D}_{d_Ad_B} $$

In other words, for any separable density matrix $\rho_{AB}\in\mathrm{SEP}$, it can be decomposed as

$$ \rho=\sum_{i\in[N]}p_{i}\left|\psi_{A,i}\right\rangle \left\langle \psi_{A,i}\right|\otimes\left|\psi_{B,i}\right\rangle \left\langle \psi_{B,i}\right| $$

$$ \begin{cases}
p_{i}\geq0\\
\sum_{i}p_{i}=1\\
\left|\psi_{A,i}\right\rangle \in\mathcal{H}_{A}\\
\left|\psi_{B,i}\right\rangle \in\mathcal{H}_{B}
\end{cases} $$

where the number of summation terms $N$ is finite, and [arxiv-link](https://arxiv.org/abs/quant-ph/9703004) proves that $N\leq d_A^2d_B^2$ according to Carathéodory theorem. As a finite convex combination, SEP is a convex compact closed set in the space of density matrices. The complement of SEP is the set of entangled states, which we will call it ENT.

$$ \mathrm{ENT}=\mathcal{D}_{d_{A}d_{B}}\setminus\mathrm{SEP} $$

Apparently, the maximally mixed state is a separable state

$$ \rho_0 = \sum_{i,j\in[d]} \frac{1}{d^2} | i \rangle\langle i |\otimes| j \rangle\langle j |.$$

## PPT criterion

However, determine whether a mixed state is entangled or not is a NP-hard problem. There are some criteria that can be used to detect entanglement, e.g. PPT criterion [wiki-link](https://en.wikipedia.org/wiki/Peres%E2%80%93Horodecki_criterion). The PPT criteria is based on the fact that the partial transpose of a separable state is still a positive semidefinite matrix , while the partial transpose of an entangled state may not. The partial transpose $\Gamma$ of a matrix $\rho$ is defined as

$$ \rho=\sum_{ijkl}\rho_{ij,kl}|ij\rangle\langle kl|\in \mathbb{C}^{d_Ad_B\times d_Ad_B}$$

$$ \rho^{\Gamma}=\sum_{ijkl}\rho_{il,kj} |ij\rangle\langle kl| $$

where we denote it with $\rho^{\Gamma}$. To check whether a matrix is positive semidefinite, we can check whether all its eigenvalues are nonnegative.

Let's check the partial transpose of the separable states

$$ \rho=\sum_{i\in[N]}p_{i}\left|\psi_{A,i}\right\rangle \left\langle \psi_{A,i}\right|\otimes\left|\psi_{B,i}\right\rangle \left\langle \psi_{B,i}\right| $$

$$ \rho^\Gamma=\sum_{i\in[N]}p_{i}\left|\psi_{A,i}\right\rangle \left\langle \psi_{A,i}\right|\otimes\mathrm{conj}(\left|\psi_{B,i}\right\rangle \left\langle \psi_{B,i}\right|) $$

where $\mathrm{conj}()$ denotes the complex conjugate. It is easy to see that the partial transpose of a separable state is still a positive semidefinite matrix. For the entangled states, Let's explicitly calculate the partial transpose of the Bell state

$$ \rho_{\mathrm{Bell}}=\frac{1}{2}\begin{bmatrix}1&0&0&1\\ 0&0&0&0\\ 0&0&0&0\\ 1&0&0&1\\\end{bmatrix} $$

and its partial transpose is

$$ \rho_{\mathrm{Bell}}^{\Gamma}=\frac{1}{2}\begin{bmatrix}1&0&0&0\\ 0&0&1&0\\ 0&1&0&0\\ 0&0&0&1\\\end{bmatrix} $$

with a negative eigenvalue $-1/2$. Therefore, the Bell state is entangled.

Let's check the above states with PPT criterion.

In [ ]:
for state in [state00, state_bell, state_ghz, state_w]:
    dimA = 2
    dimB = state.size//dimA
    dm = state.reshape(-1,1) * state.reshape(-1).conj()
    tmp0 = 'separable' if numqi.entangle.is_ppt(dm, (dimA,dimB)) else 'entangled'
    print(f'[{tmp0}]')

Let's define the set of PPT states as all states that satisfy the PPT criterion, and its complement is the set of NPT states.

$$ \mathrm{PPT}=\left\{ \rho\in\mathcal{D}_{d_{A}d_{B}}:\rho^{\Gamma}\in\mathcal{D}_{d_{A}d_{B}}\right\}  $$

The PPT set is also convex, closed subset of $\mathcal{D}_{d_Ad_B}$. Apparently, SEP set is a subset of PPT set 

$$ \mathrm{SEP}\subseteq \mathrm{PPT}\subseteq \mathcal{D}_{d_Ad_B} $$

When dimension is less or equal than six $d_Ad_B\leq 6$, $\mathrm{PPT}=\mathrm{SEP}$.

In previous tutorial [doc-link](../geometry/numerical_range_dm.ipynb), we introduce numerical range for visualizing the geometry of the density matrix by projecting all density matrix into a two dimensional plane. We can also do this for PPT set. Let's visualize the PPT set for two qubits with operators

$$ O_1=X\otimes X,O_2=Z\otimes Z. $$

In [ ]:
op0 = np.kron(numqi.gate.X, numqi.gate.X)
op1 = np.kron(numqi.gate.Z, numqi.gate.Z)
# op1 = np.kron(numqi.gate.Z, numqi.gate.I) + np.kron(numqi.gate.I, numqi.gate.Z)

theta_list = np.linspace(0, 2*np.pi, 400)
direction = np.stack([np.cos(theta_list), np.sin(theta_list)], axis=1)

beta_dm = numqi.matrix_space.get_joint_algebraic_numerical_range([op0,op1], direction)
nr_dm = beta_dm.reshape(-1,1)*direction

beta_ppt = numqi.entangle.get_ppt_numerical_range([op0, op1], direction, dim=(2,2))
nr_ppt = beta_ppt.reshape(-1,1)*direction

In [ ]:
fig,ax = plt.subplots(figsize=(8,6))
ax.plot(nr_dm[:,0], nr_dm[:,1], label='DM') #density matrix
ax.plot(nr_ppt[:,0], nr_ppt[:,1], label='PPT')
ax.legend()
ax.set_label('$O_0$')
ax.set_label('$O_1$')

Apparently, the PPT set is within density matrix set (labeled "DM" in the figure). And for this two-qubits Hilbert space, the PPT set is equavalent to the SEP set. One can check that the PPT vertex $(0,\pm 1)$ corresponds to the density matrix

$$ \frac{1}{2}|00\rangle\langle 00| + \frac{1}{2}|11\rangle\langle 11| $$

$$ \frac{1}{2}|00\rangle\langle 00| - \frac{1}{2}|11\rangle\langle 11| $$

*Task*: can you figure out the density matrix corresponds to the other two vertices $(\pm 1, 0)$?

*Task*: What about the numerical range with respect to the operators $O_1=X\otimes X,O_2=Z\otimes I+I\otimes Z $

For larger dimension $d_Ad_B>6 (d_A\geq 2, d_B\geq 2)$, the PPT criterion is a necessary condition for separability, but not sufficient, and there is PPT state which is not separable (bounded entanglement [wiki-link](https://en.wikipedia.org/wiki/Bound_entanglement)).

Below, we give an example of PPT but not separable state, called Tiles UPB/BES (Bounded Entangled state from Unextendible Product States) in two-qutrits system $\mathcal{H}_3\otimes \mathcal{H}_3$. But this state can be detected by so-called generalized PPT criteria from paper "The generalized partial transposition criterion for separability of multipartite quantum states" [doi-link](https://doi.org/10.1016/S0375-9601(02)01538-4). In the later tutorials, we will introduce more advanced entanglement detection criteria.

In [ ]:
upb_list, rho_bes = numqi.entangle.load_upb('tiles', return_bes=True)
print('satisfy PPT criterion:', numqi.entangle.is_ppt(rho_bes, dim=(3,3)))
print('satisfy generalized PPT criterion:', numqi.entangle.is_generalized_ppt(rho_bes, dim=(3,3)))

## Further reading

Some good references for entanglement detection criteria see

1. [arxiv-link](https://arxiv.org/abs/quant-ph/9703004v2) Separability criterion and inseparable mixed states with positive partial transposition, by Pawel Horodecki